In [1]:
#from google.colab import drive
#drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/nlp/trace_dataset.csv'

In [3]:
!nvidia-smi

Sat Jul  9 15:21:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install transformers
!pip install datasets
!pip install ray[tune]
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 34.8 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 6.6 MB 61.9 MB/s 
     |████████████████████████████████| 596 kB 15.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 31.6 MB/s 
     |████████████████████████████████| 212 kB 54.0 MB/s 
     |████████████████████████████████| 140 kB 71.5 MB/s 
     |████████████████████████████████| 1.1 MB 61.6 MB/s 
     |████████████████████████████████| 127 kB 64.8 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 73.8 MB/s 
     |███████████

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split




aggressions = load_dataset('csv', data_files={'train': path
                                              })
print(aggressions)
train_ds = aggressions['train']
train_ds

aggressions.set_format(type='pandas')
df = aggressions['train'][:]
df.head()

# Label encoding

dt = {'NAG':0, 'CAG':1, 'OAG':2}
df['Category'].unique()
labels = map(lambda x: dt[x], df['Category'])
labels = list(labels)
labels = pd.Series(labels)

df['Label'] = labels
labs = np.array(labels)
df.head(5)



Using custom data configuration default-ebc64cd7a2be9cf9


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ebc64cd7a2be9cf9/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Category'],
        num_rows: 12663
    })
})


,Sentence,Category,Label
0,what a wicket !!!!!!!!! woohooooo #indvsuae #w...,NAG,0
1,And because of u people there is a boundary cr...,OAG,2
2,Arandhuti Roy is a person who do not believe i...,NAG,0
3,Saurabh Shinde wht does that mean at least he ...,CAG,1
4,NBCC has a very good order book but will that ...,NAG,0


In [6]:
model_ckpt = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenizer.vocab_size

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

50265

In [7]:
texts = list(df['Sentence'])
labels = list(df['Label'])

In [8]:
train_size=0.8

# In the first step we will split the data in training and remaining dataset
train_texts, X_rem, train_labels, y_rem = train_test_split(texts, labels, train_size=0.8, random_state=43)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
valid_texts, test_texts, valid_labels, test_labels = train_test_split(X_rem, y_rem, test_size=0.5, random_state=43)

print(str(np.shape(train_texts))), print(str(np.shape(train_labels)))
print(str(np.shape(valid_texts))), print(str(np.shape(valid_labels)))
print(str(np.shape(test_texts))), print(str(np.shape(test_labels)))

(10130,)
(10130,)
(1266,)
(1266,)
(1267,)
(1267,)


(None, None)

In [9]:
# USE this only for hyperparameter tuning
# In this step we will split the train data (10 %) and validation data (20%) only for hyperparameter tuning 


_texts, hyp_texts, _labels, hyp_labels = train_test_split(train_texts, train_labels, test_size=0.05, random_state=43)
print(str(np.shape(hyp_texts))), print(str(np.shape(hyp_labels)))

_texts2, hyp_v_texts, _labels2, hyp_v_labels = train_test_split(valid_texts, valid_labels, test_size=0.1, random_state=43)
print(str(np.shape(hyp_v_texts))), print(str(np.shape(hyp_v_labels)))



(507,)
(507,)
(127,)
(127,)


(None, None)

In [10]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True)
hyp_encodings = tokenizer(hyp_texts, truncation=True, padding=True)
hyp_v_encodings = tokenizer(hyp_v_texts, truncation=True, padding=True)

In [11]:
import torch

class AggressionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
train_dataset = AggressionDataset(train_encodings, train_labels)
valid_dataset = AggressionDataset(valid_encodings, valid_labels)
hyp_dataset = AggressionDataset(hyp_encodings, hyp_labels)
hyp_v_dataset = AggressionDataset(hyp_v_encodings, hyp_v_labels)

In [13]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [14]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
device = torch.device('cuda' if 
                      torch.cuda.is_available() else 'cpu')
print(device)
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels))

cuda


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, plot_confusion_matrix

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)

  f1 = f1_score(labels, preds, average='macro')
  precision = precision_score(labels, preds, average='macro')
  recall = recall_score(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

In [16]:
import tensorflow
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [17]:

gc.collect()
torch.cuda.empty_cache()

In [18]:
from torch import nn

class CustomTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    device = torch.device('cuda')
    inputs.to(device)
    labels = inputs.get("labels")
    # forward pass
    outputs = model(**inputs)
    logits = outputs.get("logits")
    # compute custom loss (suppose one has 3 labels with different weights)
    loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.21, 0.33, 0.46]).to(device))
    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
    return (loss, outputs) if return_outputs else loss

In [19]:
from ray.tune.schedulers import PopulationBasedTraining
import ray
from ray import tune
from ray.tune import CLIReporter

In [20]:
from transformers import AutoModelForSequenceClassification

# num_labels = 3
# device = torch.device('cuda')
# if torch.cuda.is_available() else 'cpu'
# print(device)
# model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

def get_my_model():
  num_labels = 3
  device = torch.device('cuda')
  return (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

In [21]:
from datasets import load_metric

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  metric = load_metric("f1")
  return metric.compute(predictions=preds, references=labels, average="macro")

In [22]:
# See this for more stoppers
# https://docs.ray.io/en/latest/tune/api_docs/stoppers.html

from ray.tune import Stopper

class CustomStopper(tune.Stopper):
        def __init__(self):
            self.should_stop = False

        def __call__(self, trial_id, result):
            max_iter = 3 
            if not self.should_stop and result["training_iteration"] >= max_iter:
                self.should_stop = True

        def stop_all(self):
            return self.should_stop

stopper = CustomStopper()

In [23]:
from transformers import Trainer, TrainingArguments
from ray.tune.examples.pbt_transformers.utils import build_compute_metrics_fn

# task_name = 'rte'

training_args = TrainingArguments(
        output_dir=".",
        learning_rate=1e-5,  # config
        do_train=True,
        do_eval=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        num_train_epochs=2,  # config
        max_steps=-1,
        per_device_train_batch_size=8,  # config
        per_device_eval_batch_size=18,  # config
        warmup_steps=0,
        weight_decay=0.1,  # config
        logging_dir="./logs",
        skip_memory_metrics=True,
        report_to="none",
        # metric_for_best_model=metric_name,

    )

trainer = CustomTrainer(model_init=get_my_model, args=training_args,
        compute_metrics = compute_metrics,
        train_dataset = hyp_dataset,
        eval_dataset = hyp_v_dataset,
        tokenizer = tokenizer)

tune_config = {
        "per_device_train_batch_size": tune.choice([8,16]),
        "per_device_eval_batch_size": 8,
        "num_train_epochs": tune.choice([2, 3, 4, 5]),
        # "max_steps": 1 if smoke_test else -1,  # Used for smoke test.
    }

scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="eval_f1",
        mode="max",
        perturbation_interval=1,
        hyperparam_mutations={
            "weight_decay": tune.uniform(0.0, 0.3),
            "learning_rate": tune.uniform(1e-5, 5e-5),
            "per_device_train_batch_size": [16, 32],
        },
    )

reporter = CLIReporter(
        parameter_columns={
            "weight_decay": "w_decay",
            "learning_rate": "lr",
            "per_device_train_batch_size": "train_bs/gpu",
            "num_train_epochs": "num_epochs",
        },
        metric_columns=["eval_f1", "eval_loss", "epoch", "training_iteration"],
    )


ress = trainer.hyperparameter_search(
        hp_space=lambda _: tune_config,
        backend="ray",
        n_trials=3,
        resources_per_trial={"cpu": 0, "gpu": 1},
        scheduler=scheduler,
        keep_checkpoints_num=1,
        checkpoint_score_attr="training_iteration",
        stop=stopper,
        progress_reporter=reporter,
        local_dir="~/ray_results/",
        name="tune_transformer_pbt",
        log_to_file=True,
    )

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transfo

== Status ==
Current time: 2022-07-09 15:23:00 (running for 00:00:00.19)
Memory usage on this node: 5.0/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/tune_transformer_pbt
Number of trials: 3/3 (2 PENDING, 1 RUNNING)
+------------------------+----------+----------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc            |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+----------------+-----------+-------------+----------------+--------------|
| _objective_04ad6_00000 | RUNNING  | 172.28.0.2:618 | 0.285214  | 3.92798e-05 |              8 |            5 |
| _objective_04ad6_00001 | PENDING  |                | 0.0468056 | 1.62398e-05 |              8 |            2 |
| _objective_04ad6_00002 | PENDING  |                | 0.25

(_objective pid=618) Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias']
(_objective pid=618) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=618) - This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=618) Some weights of RobertaForSequenceClassification were not initialized from the model checkpoi

== Status ==
Current time: 2022-07-09 15:23:20 (running for 00:00:19.85)
Memory usage on this node: 8.7/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/tune_transformer_pbt
Number of trials: 3/3 (2 PENDING, 1 RUNNING)
+------------------------+----------+----------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc            |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+----------------+-----------+-------------+----------------+--------------|
| _objective_04ad6_00000 | RUNNING  | 172.28.0.2:618 | 0.285214  | 3.92798e-05 |              8 |            5 |
| _objective_04ad6_00001 | PENDING  |                | 0.0468056 | 1.62398e-05 |              8 |            2 |
| _objective_04ad6_00002 | PENDING  |                | 0.25

  7%|▋         | 22/320 [00:09<02:04,  2.39it/s]


...

== Status ==
Current time: 2022-07-09 15:30:49 (running for 00:07:48.74)
Memory usage on this node: 9.7/12.7 GiB
PopulationBasedTraining: 2 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/tune_transformer_pbt
Number of trials: 3/3 (2 ERROR, 1 RUNNING)
+------------------------+----------+-----------------+-----------+-------------+----------------+--------------+-----------+-------------+---------+----------------------+
| Trial name             | status   | loc             |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+-----------------+-----------+-------------+----------------+--------------+-----------+-------------+---------+----------------------|
| _objective_04ad6_00001 | RUNNING  | 172.28.0.2:1340 | 0.0468056 | 1.62398e-05 |              8 |         

100%|██████████| 128/128 [01:14<00:00,  2.56it/s]
(_objective pid=1340) 
  0%|          | 0/16 [00:00<?, ?it/s]
(_objective pid=1340) 
 12%|█▎        | 2/16 [00:00<00:00, 16.81it/s]
(_objective pid=1340) 
 25%|██▌       | 4/16 [00:00<00:01, 10.98it/s]
(_objective pid=1340) 
 38%|███▊      | 6/16 [00:00<00:01,  9.75it/s]
(_objective pid=1340) 
 50%|█████     | 8/16 [00:00<00:00,  9.26it/s]
(_objective pid=1340) 
 56%|█████▋    | 9/16 [00:00<00:00,  9.17it/s]
(_objective pid=1340) 
 62%|██████▎   | 10/16 [00:01<00:00,  9.13it/s]
(_objective pid=1340) 
 69%|██████▉   | 11/16 [00:01<00:00,  9.02it/s]
(_objective pid=1340) 
 75%|███████▌  | 12/16 [00:01<00:00,  8.96it/s]
(_objective pid=1340) 
 81%|████████▏ | 13/16 [00:01<00:00,  8.95it/s]
(_objective pid=1340) 
 88%|████████▊ | 14/16 [00:01<00:00,  8.84it/s]
(_objective pid=1340) 
 94%|█████████▍| 15/16 [00:01<00:00,  8.84it/s]
(_objective pid=1340) 
100%|██████████| 16/16 [00:01<00:00,  8.92it/s]
                                         

(_objective pid=1340) {'eval_loss': 0.9769155979156494, 'eval_f1': 0.43532417757904823, 'eval_runtime': 2.8308, 'eval_samples_per_second': 44.864, 'eval_steps_per_second': 5.652, 'epoch': 2.0}
== Status ==
Current time: 2022-07-09 15:30:54 (running for 00:07:53.75)
Memory usage on this node: 9.7/12.7 GiB
PopulationBasedTraining: 2 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/tune_transformer_pbt
Number of trials: 3/3 (2 ERROR, 1 RUNNING)
+------------------------+----------+-----------------+-----------+-------------+----------------+--------------+-----------+-------------+---------+----------------------+
| Trial name             | status   | loc             |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+-----------------+-----------+-------------+------

100%|██████████| 128/128 [01:32<00:00,  1.38it/s]


TuneError: ignored